# Enrichr Hypergraphs

In [1]:
import requests
import time
from tqdm import tqdm
from HAT import Hypergraph
from datetime import datetime
import json
import numpy as np
import os
import math
from pathlib import Path

In [2]:
def convert_numpy(obj):
    if isinstance(obj, dict):
        return {k: convert_numpy(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy(i) for i in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

def get_file_size_mb(filepath):
    """Get file size in MB"""
    return os.path.getsize(filepath) / (1024 * 1024)

def count_lines_in_file(filepath):
    """Count total lines in a file"""
    with open(filepath, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

def split_file_by_lines(filepath, max_size_mb=100):
    """Split a file into chunks based on line count"""
    print(f"Processing: {filepath}")
    
    # Check if file needs splitting
    file_size_mb = get_file_size_mb(filepath)
    if file_size_mb <= max_size_mb:
        print(f"  File is already under {max_size_mb}MB ({file_size_mb:.2f}MB), skipping...")
        return
    
    # Calculate number of chunks needed
    num_chunks = math.ceil(file_size_mb / max_size_mb)
    print(f"  File size: {file_size_mb:.2f}MB")
    print(f"  Number of chunks needed: {num_chunks}")
    
    # Count total lines
    total_lines = count_lines_in_file(filepath)
    print(f"  Total lines: {total_lines}")
    
    # Calculate lines per chunk
    lines_per_chunk = math.ceil(total_lines / num_chunks)
    print(f"  Lines per chunk: {lines_per_chunk}")
    
    # Get the base name without extension
    base_name = Path(filepath).stem
    
    # Split the file
    chunk_files = []
    with open(filepath, 'r', encoding='utf-8') as input_file:
        chunk_num = 1
        lines_written = 0
        
        while True:
            # Create chunk filename
            chunk_filename = f"{base_name}_{chunk_num}of{num_chunks}.hif"
            chunk_filepath = Path(filepath).parent / chunk_filename
            
            # Write lines to current chunk
            with open(chunk_filepath, 'w', encoding='utf-8') as chunk_file:
                for _ in range(lines_per_chunk):
                    line = input_file.readline()
                    if not line:  # End of file
                        break
                    chunk_file.write(line)
                    lines_written += 1
            
            # Check if we wrote anything to this chunk
            if lines_written == 0:
                # Remove empty chunk file if created
                if chunk_filepath.exists():
                    os.remove(chunk_filepath)
                break
            
            chunk_size = get_file_size_mb(chunk_filepath)
            chunk_files.append(chunk_filename)
            print(f"  Created: {chunk_filename} ({chunk_size:.2f}MB, {min(lines_per_chunk, total_lines - (chunk_num-1)*lines_per_chunk)} lines)")
            
            chunk_num += 1
            
            # Break if we've processed all lines
            if lines_written >= total_lines:
                break
    
    # Update the actual number of chunks created
    actual_chunks = len(chunk_files)
    if actual_chunks != num_chunks:
        print(f"  Note: Created {actual_chunks} chunks instead of estimated {num_chunks}")
        
        # Rename files to reflect actual count
        for i, old_filename in enumerate(chunk_files, 1):
            old_filepath = Path(filepath).parent / old_filename
            new_filename = f"{base_name}_{i}of{actual_chunks}.hif"
            new_filepath = Path(filepath).parent / new_filename
            
            if old_filepath != new_filepath:
                os.rename(old_filepath, new_filepath)
                print(f"  Renamed: {old_filename} -> {new_filename}")
    
    # Remove original file after successful splitting
    try:
        os.remove(filepath)
        print(f"  Removed original file: {filepath}")
    except Exception as e:
        print(f"  Warning: Could not remove original file: {e}")
    
    print(f"  Split complete: {actual_chunks} files created")

def rejoin_files(base_name, directory="."):
    """Rejoin split files back into original file"""
    directory = Path(directory)
    
    # Find all chunks for this base name
    pattern = f"{base_name}_*of*.hif"
    chunk_files = list(directory.glob(pattern))
    
    if not chunk_files:
        print(f"No chunk files found for {base_name}")
        return
    
    # Sort chunks by number
    def get_chunk_number(filename):
        # Extract number from filename like "name_1of3.hif"
        stem = filename.stem
        if "_" in stem and "of" in stem:
            try:
                number_part = stem.split("_")[-1]  # Get "1of3"
                return int(number_part.split("of")[0])  # Get "1"
            except:
                return 0
        return 0
    
    chunk_files.sort(key=get_chunk_number)
    
    print(f"Rejoining {len(chunk_files)} chunks for {base_name}")
    
    # Rejoin files
    output_file = directory / f"{base_name}.hif"
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for chunk_file in chunk_files:
            print(f"  Adding: {chunk_file.name}")
            with open(chunk_file, 'r', encoding='utf-8') as infile:
                outfile.write(infile.read())
    
    print(f"  Rejoined file created: {output_file}")
    final_size = get_file_size_mb(output_file)
    print(f"  Final size: {final_size:.2f}MB")


In [3]:
# Step 1: Get all valid gene set libraries
stats_url = "https://maayanlab.cloud/Enrichr/datasetStatistics"
response = requests.get(stats_url)
libraries = response.json()

# Step 2: Construct base URL and all full URLs
base_url = "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName="
library_names = [entry['libraryName'] for entry in libraries['statistics']]
urls = [f"{base_url}{name}" for name in library_names]
print("Steps 1-2 Complete")

Steps 1-2 Complete


In [5]:
for network_name, url in tqdm(zip(library_names, urls)):
    try:
        # Download Data from Enrichr
        r = requests.get(url)
        r.raise_for_status()  # Raise an error for bad responses
        network_text = r.text
        print(f"Downloaded: {network_name}")

        # Convert to HAT Hypergraph
        hyperedge_texts = network_text.split("\n")
        hyperedges = [] #{}
        for hyperedge_text in hyperedge_texts:
            if len(hyperedge_text) == 0:
                continue
            hyperedge_split = hyperedge_text.split("\t")
            hyperedge_split = [f for f in hyperedge_split if f != '']
            hyperedge_name  = hyperedge_split[0]
            hyperedge_nodes = hyperedge_split[1:]
            hyperedges.append(hyperedge_nodes)
        H = Hypergraph(edge_list=hyperedges)

        # Convert to HIF, add metadata, make JSON writable
        hif = H.to_hif()
        hif['metadata']['name'] = network_name
        hif['metadata']['source'] = "Enrichr: " + url
        hif['metadata']['date'] = datetime.now().strftime("%Y-%m-%d")
        hif_clean = convert_numpy(hif)

        # Write to a file
        hif_file_name = "/nfs/turbo/umms-indikar/Joshua/HIF-datasets/datasets/" + network_name + ".hif"
        with open(hif_file_name, "w") as f:
            json.dump(hif_clean, f, indent=4)
        print("    Written to File")

        # Convert to split files
        split_file_by_lines(hif_file_name, max_size_mb=40)
        print("    Split to separate files...Complete ✓")
    
    except requests.exceptions.RequestException as e:
        print(f"✗ Failed to download {name}: {e}")


0it [00:00, ?it/s]

Downloaded: Genome_Browser_PWMs


1it [00:04,  4.84s/it]

    Written to File
Processing: /nfs/turbo/umms-indikar/Joshua/HIF-datasets/datasets/Genome_Browser_PWMs.hif
  File is already under 40MB (27.92MB), skipping...
    Split to separate files...Complete ✓
Downloaded: TRANSFAC_and_JASPAR_PWMs
    Written to File
Processing: /nfs/turbo/umms-indikar/Joshua/HIF-datasets/datasets/TRANSFAC_and_JASPAR_PWMs.hif
  File size: 68.64MB
  Number of chunks needed: 2
  Total lines: 3504593
  Lines per chunk: 1752297
  Created: TRANSFAC_and_JASPAR_PWMs_1of2.hif (35.55MB, 1752297 lines)


2it [00:32, 18.21s/it]

...

  Created: TISSUES_Curated_2025_2of2.hif (21.26MB, 1128361 lines)
  Removed original file: /nfs/turbo/umms-indikar/Joshua/HIF-datasets/datasets/TISSUES_Curated_2025.hif
  Split complete: 2 files created
    Split to separate files...Complete ✓
Downloaded: RummaGEO_DrugPerturbations_2025
    Written to File
Processing: /nfs/turbo/umms-indikar/Joshua/HIF-datasets/datasets/RummaGEO_DrugPerturbations_2025.hif
  File size: 640.21MB
  Number of chunks needed: 17
  Total lines: 32287737
  Lines per chunk: 1899279
  Created: RummaGEO_DrugPerturbations_2025_1of17.hif (46.99MB, 1899279 lines)
  Created: RummaGEO_DrugPerturbations_2025_2of17.hif (47.71MB, 1899279 lines)
  Created: RummaGEO_DrugPerturbations_2025_3of17.hif (37.99MB, 1899279 lines)
  Created: RummaGEO_DrugPerturbations_2025_4of17.hif (35.81MB, 1899279 lines)
  Created: RummaGEO_DrugPerturbations_2025_5of17.hif (35.81MB, 1899279 lines)
  Created: RummaGEO_DrugPerturbations_2025_6of17.hif (36.25MB, 1899279 lines)
  Created: RummaGEO_

216it [51:13, 43.46s/it]

  Created: RummaGEO_DrugPerturbations_2025_17of17.hif (36.33MB, 1899273 lines)
  Removed original file: /nfs/turbo/umms-indikar/Joshua/HIF-datasets/datasets/RummaGEO_DrugPerturbations_2025.hif
  Split complete: 17 files created
    Split to separate files...Complete ✓
Downloaded: RummaGEO_GenePerturbations_2025
    Written to File
Processing: /nfs/turbo/umms-indikar/Joshua/HIF-datasets/datasets/RummaGEO_GenePerturbations_2025.hif
  File size: 483.33MB
  Number of chunks needed: 13
  Total lines: 24385897
  Lines per chunk: 1875839
  Created: RummaGEO_GenePerturbations_2025_1of13.hif (46.39MB, 1875839 lines)
  Created: RummaGEO_GenePerturbations_2025_2of13.hif (43.37MB, 1875839 lines)
  Created: RummaGEO_GenePerturbations_2025_3of13.hif (35.37MB, 1875839 lines)
  Created: RummaGEO_GenePerturbations_2025_4of13.hif (35.37MB, 1875839 lines)
  Created: RummaGEO_GenePerturbations_2025_5of13.hif (35.76MB, 1875839 lines)
  Created: RummaGEO_GenePerturbations_2025_6of13.hif (35.88MB, 1875839 li

217it [52:50, 14.61s/it]

  Created: RummaGEO_GenePerturbations_2025_13of13.hif (35.88MB, 1875829 lines)
  Removed original file: /nfs/turbo/umms-indikar/Joshua/HIF-datasets/datasets/RummaGEO_GenePerturbations_2025.hif
  Split complete: 13 files created
    Split to separate files...Complete ✓


# Scratch

In [13]:
# library_names = [entry['libraryName'] for entry in data]

# Generate all URLs like the one you posted
base_url = "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName="
urls = [f"{base_url}{name}" for name in library_names]

# Print the first 5 as example
for u in urls[:5]:
    print(u)


https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=Genome_Browser_PWMs
https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=TRANSFAC_and_JASPAR_PWMs
https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=Transcription_Factor_PPIs
https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=ChEA_2013
https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=Drug_Perturbations_from_GEO_2014
